In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.utils import Sequence
import pickle
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, GlobalAveragePooling2D, GlobalMaxPooling2D, Flatten, Concatenate
import orjson
from threading import Thread, Lock
from queue import Queue

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('../dataset_test_files.pkl', 'rb') as fp:
    df_test = pickle.load(fp)
df_test.head()

,filename,kind,defect,size,class_name,class
3,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
4,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
11,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
15,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
20,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [3]:
with open('../dataset_train_files.pkl', 'rb') as fp:
    df_train = pickle.load(fp)
df_train.head()

,filename,kind,defect,size,class_name,class
0,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
1,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
2,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
5,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
6,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [4]:
class ImageGeneratorSplited(Sequence):
    def __init__(self, df, y_col='class_name', batch_size=64, split=0.5, queue_size=100, **kwargs):
        self.gen_da = ImageDataGenerator(**kwargs)
        self.gen_raw = ImageDataGenerator()
        self.length = int(len(df.index) / (batch_size * (1 - split)))
        self.batch_size = batch_size
        self.batch_da = int(batch_size * split)
        self.flow_da = self.gen_da.flow_from_dataframe(df, y_col=y_col, batch_size=self.batch_da)
        self.batch_raw = self.batch_size - self.batch_da
        self.flow_raw = self.gen_raw.flow_from_dataframe(df, y_col=y_col, batch_size=self.batch_raw)
        temp = next(self.flow_raw)
        self.shape = (self.length, *temp[0].shape[1:])
        self._next_da = Queue(queue_size)
        self._next_raw = Queue(queue_size)
        self._next_da_thread = self.prepare_da()
        self._next_raw_thread = self.prepare_raw()
        
    def __len__(self):
        return self.length
        
    def get_next_da(self):
        return self._next_da.get()
    
    def get_next_raw(self):
        return self._next_raw.get()
    
    def _prepare_next_da(self):
        while True:
            self._next_da.put(next(self.flow_da))
    
    def _prepare_next_raw(self):
        while True:
            self._next_raw.put(next(self.flow_raw))
            
    def prepare_da(self):
        t = Thread(target=self._prepare_next_da)
        t.start()
        return t
    
    def prepare_raw(self):
        t = Thread(target=self._prepare_next_raw)
        t.start()
        return t
    
    
    def __iter__(self):
        for _ in range(len(self)):
            yield self[None]

    def __getitem__(self, item):
        x_raw, y_raw = self.get_next_raw()
        x_da, y_da = self.get_next_da()
        return preprocess_input(np.vstack((x_raw, x_da))), np.vstack((y_raw, y_da))
    

In [5]:
SPLIT = 0.7
gen_train = ImageGeneratorSplited(df_train,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # zoom_range=[1, 3],
    horizontal_flip=True,
    vertical_flip=True,
    split=SPLIT,
    batch_size=64
)
print(gen_train)
gen_test = ImageGeneratorSplited(df_test, batch_size=128)
gen_test

Found 840 validated image filenames belonging to 4 classes.
Found 840 validated image filenames belonging to 4 classes.
Found 360 validated image filenames belonging to 4 classes.
Found 360 validated image filenames belonging to 4 classes.


In [6]:
SHAPE = (256, 256, 3)
input_tensor = Input(shape=SHAPE)
base_model = InceptionResNetV2(False, None, input_tensor=input_tensor)
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

In [7]:
x = base_model.output
out = Flatten()(x)
out = Dropout(0.5)(out)
predictions = Dense(4, activation="softmax", name="3_")(out)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [8]:
model.compile(optimizer='adam', # Rotina de otimização, que informa a ml como ajustar o valor dos parâmetros para minimizar o erro.
              loss='categorical_crossentropy', # Função de erro que diz o quão erradas estão nossas predições
              metrics=['accuracy']) # Lista de métricas para avaliar o nosso modelo

In [9]:
with open('model_da_resnet_inception.json', 'w') as fp:
    fp.write(model.to_json())

In [10]:
history = model.fit(
    gen_train, 
    epochs=100, 
    verbose=2, 
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_resnet_inception.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs_da_resnet_inception'),
])

Train for 43 steps, validate for 5 steps
Epoch 1/100
43/43 - 62s - loss: 4.1719 - accuracy: 0.3155 - val_loss: 1434.8781 - val_accuracy: 0.2432
Epoch 2/100
43/43 - 28s - loss: 1.4689 - accuracy: 0.4933 - val_loss: 133872.8016 - val_accuracy: 0.2449
Epoch 3/100
43/43 - 27s - loss: 2.6690 - accuracy: 0.4027 - val_loss: 74547.2727 - val_accuracy: 0.2419
Epoch 4/100
43/43 - 27s - loss: 3.6898 - accuracy: 0.3226 - val_loss: 1004109708.8000 - val_accuracy: 0.2435
Epoch 5/100
43/43 - 27s - loss: 2.5264 - accuracy: 0.3544 - val_loss: 21944.7473 - val_accuracy: 0.2230
Epoch 6/100
43/43 - 27s - loss: 2.7972 - accuracy: 0.3419 - val_loss: 13418674.0000 - val_accuracy: 0.2601
Epoch 7/100
43/43 - 27s - loss: 2.1497 - accuracy: 0.3552 - val_loss: 70180.5977 - val_accuracy: 0.2500
Epoch 8/100
43/43 - 28s - loss: 1.4319 - accuracy: 0.3952 - val_loss: 2.1771 - val_accuracy: 0.2399
Epoch 9/100
43/43 - 28s - loss: 1.1635 - accuracy: 0.4521 - val_loss: 1.5873 - val_accuracy: 0.3344
Epoch 10/100
43/43 - 28

In [11]:
with open('train_da_history.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history.history.items()}))

In [12]:
history2 = model.fit(
    gen_train, 
    epochs=130, 
    verbose=2,
    initial_epoch=100,
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_resnet_inception.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs2_da_resnet_inception'),
])

Train for 43 steps, validate for 5 steps
Epoch 101/130
43/43 - 28s - loss: 0.5825 - accuracy: 0.7717 - val_loss: 0.9383 - val_accuracy: 0.6575
Epoch 102/130
43/43 - 27s - loss: 0.6027 - accuracy: 0.7833 - val_loss: 0.8176 - val_accuracy: 0.6964
Epoch 103/130
43/43 - 26s - loss: 0.5734 - accuracy: 0.7762 - val_loss: 1.3168 - val_accuracy: 0.5929
Epoch 104/130
43/43 - 26s - loss: 0.5472 - accuracy: 0.7823 - val_loss: 0.8898 - val_accuracy: 0.6807
Epoch 105/130
43/43 - 27s - loss: 0.5459 - accuracy: 0.7945 - val_loss: 1.1173 - val_accuracy: 0.6081
Epoch 106/130
43/43 - 26s - loss: 0.5540 - accuracy: 0.7796 - val_loss: 0.8928 - val_accuracy: 0.6503
Epoch 107/130
43/43 - 27s - loss: 0.6177 - accuracy: 0.7650 - val_loss: 3.1162 - val_accuracy: 0.3929
Epoch 108/130
43/43 - 27s - loss: 0.5307 - accuracy: 0.7885 - val_loss: 1.2697 - val_accuracy: 0.6429
Epoch 109/130
43/43 - 27s - loss: 0.5233 - accuracy: 0.7975 - val_loss: 1.3749 - val_accuracy: 0.6250
Epoch 110/130
43/43 - 27s - loss: 0.5357 

In [13]:
with open('train_da_history2.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history2.history.items()}))

In [14]:
history3 = model.fit(
    gen_train, 
    epochs=200, 
    verbose=2,
    initial_epoch=130,
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_resnet_inception.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs2_da_resnet_inception'),
])

Train for 43 steps, validate for 5 steps
Epoch 131/200
43/43 - 29s - loss: 0.4986 - accuracy: 0.7998 - val_loss: 0.8941 - val_accuracy: 0.6791
Epoch 132/200
43/43 - 27s - loss: 0.4277 - accuracy: 0.8394 - val_loss: 0.6993 - val_accuracy: 0.7264
Epoch 133/200
43/43 - 27s - loss: 0.4098 - accuracy: 0.8361 - val_loss: 0.7430 - val_accuracy: 0.7208
Epoch 134/200
43/43 - 27s - loss: 0.4205 - accuracy: 0.8379 - val_loss: 0.8103 - val_accuracy: 0.7500
Epoch 135/200
43/43 - 27s - loss: 0.4151 - accuracy: 0.8327 - val_loss: 0.7797 - val_accuracy: 0.7703
Epoch 136/200
43/43 - 26s - loss: 0.3705 - accuracy: 0.8526 - val_loss: 0.8479 - val_accuracy: 0.7162
Epoch 137/200
43/43 - 27s - loss: 0.3981 - accuracy: 0.8368 - val_loss: 0.7311 - val_accuracy: 0.7736
Epoch 138/200
43/43 - 27s - loss: 0.4133 - accuracy: 0.8353 - val_loss: 0.9800 - val_accuracy: 0.6807
Epoch 139/200
43/43 - 28s - loss: 0.3875 - accuracy: 0.8563 - val_loss: 0.8264 - val_accuracy: 0.7192
Epoch 140/200
43/43 - 27s - loss: 0.3703 

In [15]:
with open('train_da_history2.json', 'wb') as fp:
        fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history2.history.items()}))

In [16]:
history3 = model.fit(
    gen_train, 
    epochs=250, 
    verbose=2,
    initial_epoch=200,
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_resnet_inception.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs2_da_resnet_inception'),
])

Train for 43 steps, validate for 5 steps
Epoch 201/250
43/43 - 28s - loss: 0.1269 - accuracy: 0.9588 - val_loss: 1.0130 - val_accuracy: 0.7760
Epoch 202/250
43/43 - 26s - loss: 0.1599 - accuracy: 0.9479 - val_loss: 0.7795 - val_accuracy: 0.7695
Epoch 203/250
43/43 - 27s - loss: 0.2014 - accuracy: 0.9356 - val_loss: 0.9417 - val_accuracy: 0.7956
Epoch 204/250
43/43 - 26s - loss: 0.1268 - accuracy: 0.9547 - val_loss: 1.2548 - val_accuracy: 0.6926
Epoch 205/250
43/43 - 26s - loss: 0.1320 - accuracy: 0.9555 - val_loss: 1.2989 - val_accuracy: 0.7635
Epoch 206/250
43/43 - 26s - loss: 0.2181 - accuracy: 0.9266 - val_loss: 1.4681 - val_accuracy: 0.6419
Epoch 207/250
43/43 - 26s - loss: 0.1993 - accuracy: 0.9382 - val_loss: 2.2499 - val_accuracy: 0.6445
Epoch 208/250
43/43 - 26s - loss: 0.1443 - accuracy: 0.9479 - val_loss: 3.0904 - val_accuracy: 0.5601
Epoch 209/250
43/43 - 26s - loss: 0.1045 - accuracy: 0.9581 - val_loss: 1.7618 - val_accuracy: 0.6943
Epoch 210/250
43/43 - 28s - loss: 0.1245 

In [18]:
with open('train_da_history4.json', 'wb') as fp:
        fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history3.history.items()}))

In [19]:
history4 = model.fit(
    gen_train, 
    epochs=300, 
    verbose=2,
    initial_epoch=250,
    validation_data=gen_test, 
    callbacks=[
    ModelCheckpoint(
        'model_da_resnet_inception.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs2_da_resnet_inception'),
])

Train for 43 steps, validate for 5 steps
Epoch 251/300
43/43 - 28s - loss: 0.8331 - accuracy: 0.7923 - val_loss: 3.1853 - val_accuracy: 0.4756
Epoch 252/300
43/43 - 27s - loss: 0.6210 - accuracy: 0.8035 - val_loss: 9.5312 - val_accuracy: 0.4464
Epoch 253/300
43/43 - 28s - loss: 0.3885 - accuracy: 0.8623 - val_loss: 3.4584 - val_accuracy: 0.5895
Epoch 254/300
43/43 - 27s - loss: 0.2303 - accuracy: 0.9103 - val_loss: 2.6711 - val_accuracy: 0.4713
Epoch 255/300
43/43 - 27s - loss: 0.2143 - accuracy: 0.9266 - val_loss: 1.2351 - val_accuracy: 0.7044
Epoch 256/300
43/43 - 26s - loss: 0.1369 - accuracy: 0.9510 - val_loss: 1.4303 - val_accuracy: 0.6605
Epoch 257/300
43/43 - 27s - loss: 0.1110 - accuracy: 0.9659 - val_loss: 0.9951 - val_accuracy: 0.7127
Epoch 258/300
43/43 - 26s - loss: 0.2428 - accuracy: 0.9173 - val_loss: 2.0065 - val_accuracy: 0.5081
Epoch 259/300
43/43 - 28s - loss: 0.1900 - accuracy: 0.9244 - val_loss: 0.8960 - val_accuracy: 0.7449
Epoch 260/300
43/43 - 27s - loss: 0.1205 

In [20]:
with open('train_da_history5.json', 'wb') as fp:
        fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history4.history.items()}))